# 1导入数据分析工具包

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import warnings
warnings.filterwarnings("ignore")
 
%matplotlib inline

In [ ]:
import numpy as np
print(np.__version__)


# 2数据读取

In [ ]:
train_data_file = "./data/zhengqi_train.txt"
test_data_file =  "./data/zhengqi_test.txt"

train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8')
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8')

# 3训练数据总览

In [ ]:
train_data.describe()

# 4特征工程

## 4.1异常值分析（箱型图）

In [ ]:
plt.figure(figsize=(18, 10))
plt.boxplot(x=train_data.values,labels=train_data.columns)
plt.hlines([-7.5, 7.5], 0, 40, colors='r') # 0,40：水平线起始结束位置
plt.show()

从箱线图可以看出，有些特征存在明显的异常值、如V9变量。接下来分别把训练集和测试集中的异常值删除。

#### 代码详解：
1. `plt.boxplot(x=train_data.values,labels=train_data.columns)`：使用`boxplot`函数绘制箱线图。其中，`train_data.values`是要绘制箱线图的数据，`labels=train_data.columns`表示箱线图中每个箱子对应的标签是`train_data`数据集的列名。


2. `plt.hlines([-7.5, 7.5], 0, 40, colors='r')`：使用`hlines`函数绘制水平线。`[-7.5, 7.5]`表示要绘制的水平线的位置，0和40分别表示水平线的起始和结束位置，`colors='r'`表示线条颜色为红色。

该段代码使用matplotlib库绘制了一个箱线图，展示了`train_data`数据集中各个特征列的分布情况，并在图中添加了两条水平线（-7.5和7.5），用于标记异常值的阈值。

### 4.1.1删除异常值

In [ ]:
train_data = train_data[train_data['V9']>-7.5] # 保留大于-7.5的值
test_data = test_data[test_data['V9']>-7.5]

display(train_data.describe())
display(test_data.describe())

## 4.2特征缩放（最大最小值归一化）

In [ ]:
from sklearn import preprocessing 

features_columns = [col for col in train_data.columns if col not in ['target']]

min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = min_max_scaler.fit(train_data[features_columns])

train_data_scaler = min_max_scaler.transform(train_data[features_columns])
test_data_scaler = min_max_scaler.transform(test_data[features_columns])

train_data_scaler = pd.DataFrame(train_data_scaler)
train_data_scaler.columns = features_columns

test_data_scaler = pd.DataFrame(test_data_scaler)
test_data_scaler.columns = features_columns

train_data_scaler['target'] = train_data['target']

In [ ]:
display(train_data_scaler.describe())
display(test_data_scaler.describe())

#### 代码详解：

1. `features_columns = [col for col in train_data.columns if col not in ['target']]`：使用列表推导式创建一个列表`features_columns`，其中包含除了'target'列之外的所有训练数据的特征列。

3. `min_max_scaler = preprocessing.MinMaxScaler()`：创建一个MinMaxScaler对象，用于进行特征缩放。

4. `min_max_scaler = min_max_scaler.fit(train_data[features_columns])`：使用训练数据的特征列拟合（fit）MinMaxScaler对象，计算训练集中每个特征的最小值和最大值。

5. `train_data_scaler = min_max_scaler.transform(train_data[features_columns])`：将训练数据的特征列使用MinMaxScaler进行缩放转换，得到缩放后的训练数据。

6. `test_data_scaler = min_max_scaler.transform(test_data[features_columns])`：将测试数据的特征列使用相同的MinMaxScaler进行缩放转换，得到缩放后的测试数据。这里使用的是在训练数据上拟合的MinMaxScaler对象，保证了训练数据和测试数据的缩放方式一致。

7. `train_data_scaler = pd.DataFrame(train_data_scaler)`：将缩放后的训练数据转换为DataFrame格式。

8. `train_data_scaler.columns = features_columns`：将缩放后的训练数据的列名设置为原始特征列的列名，保持一致性。

9. `test_data_scaler = pd.DataFrame(test_data_scaler)`：将缩放后的测试数据转换为DataFrame格式。

10. `test_data_scaler.columns = features_columns`：将缩放后的测试数据的列名设置为原始特征列的列名，保持一致性。

11. `train_data_scaler['target'] = train_data['target']`：将缩放后的训练数据中的'target'列设置为原始训练数据的'target'列，这样保留了目标变量。

-----
**fit与transform：**

1. fit方法：
   - fit方法用于从训练数据中学习模型的参数或数据转换所需的统计信息。
   - 在fit方法中，模型会根据训练数据自动计算并确定转换所需的参数，例如均值、标准差等。
   - fit方法通常只针对训练数据集进行调用，以便使模型能够根据训练数据适应最佳的参数设置。

2. transform方法：
   - transform方法用于将原始数据转换为经过预处理后的数据。
   - 在transform方法中，模型会根据之前学习到的参数或统计信息，对数据进行相应的转换操作。
   - transform方法通常在训练数据和测试数据上分别调用，以便将它们都转换为相同的预处理格式，保证数据的一致性。

fit和transform方法在不同的预处理类中具有相对应的方法名：

- MinMaxScaler类，fit方法用于计算每个特征的最小值和最大值，transform方法用于将数据缩放到给定的范围，默认为[0, 1]。

- StandardScaler类，fit方法用于计算每个特征的均值和标准差，transform方法用于将数据进行标准化处理，使其均值为0，标准差为1。

- OneHotEncoder类，fit方法用于确定类别特征的所有可能取值，并构建映射关系，transform方法用于将类别特征转换为二进制矩阵表示。

fit方法用于从 ==训练数据== 中 ==学习== 模型的 ==参数== 或统计信息，transform方法用于将原始数据按照之前学习到的参数或统计信息进行转换。在实际应用中，通常先调用fit方法对模型进行训练，然后再调用transform方法对训练数据和测试数据进行相同的数据预处理操作。

使用MinMaxScaler对数据进行缩放时，通常将其fit在训练集上的原因是为了避免信息泄漏（information leakage）。

信息泄漏是指在数据预处理过程中，使用了不应该在当前步骤中可用的额外信息。这可能导致模型在实际应用中性能下降，因为会在测试集上产生过于乐观的估计。

将MinMaxScaler仅拟合（fit）于训练数据的优势有两个：
- 模型只能使用训练集的信息进行训练和预测，这符合机器学习的基本原则。模型无法直接访问测试集的任何信息，以确保其在真实世界中的性能。
- 防止信息泄漏。如果将MinMaxScaler同时拟合于训练集和测试集，就相当于使用了测试集的信息来进行预处理，将测试集的信息引入到了训练过程中。这可能会导致过于乐观的结果。
- 测试集是用于评估模型性能的独立数据集。如果对测试集拟合（fit），则使用不同的预处理参数，那么测试集的数据范围和分布就会与训练集不一致，这可能导致模型在实际应用中表现不佳。


------
**`column = train_data.columns.tolist()[:39]`与
`columns = [col for col in train_data.columns]`
这两种获取列名的区别和优劣：** 

1. `column = train_data.columns.tolist()[:39]`：
   - 这种方式将DataFrame的列名转换为一个Python列表。
   - 优点：简单直接，获取指定数量的列名。
   - 缺点：只能获取指定数量的列名，不适用于需要选择特定列或基于某些条件筛选列名的情况。

2. `columns = [col for col in train_data.columns]`：
   - 这种方式使用列表推导式将DataFrame的所有列名存储到一个列表中。
   - 优点：可以获取DataFrame中的所有列名，提供了更大的灵活性。可以通过条件筛选、修改等方式来操作列名。
   - 缺点：相对于第一种方式来说，代码稍微复杂一些。

所以，选择哪种方式取决于具体的需求：
- 如果只需要获取特定数量的列名，且数量固定，可以使用`train_data.columns.tolist()[:39]`方式，简单且明确。
- 如果需要对列名进行更多的操作，如条件筛选、修改等，或者需要获取所有列名，例如`col for col in train_data.columns if col not in ['target']` 那么使用列表推导式会更灵活。



## 4.3查看数据分布情况

我们利用KDE分布对比特征变量在训练集和测试集中的分布情况。对比后发现：特征变量V5, V9, V11, V17, V22, V28在训练集和测试集中的分布差异较大，会影响模型的泛化能力，故删除这些特征

In [ ]:
# KDE分布
dist_cols = 6
dist_rows = len(test_data_scaler.columns)

plt.figure(figsize=(4*dist_cols,4*dist_rows))


for i, col in enumerate(test_data_scaler.columns):
    ax=plt.subplot(dist_rows,dist_cols,i+1)
    ax = sns.kdeplot(train_data_scaler[col], color="Red", shade=True)
    ax = sns.kdeplot(test_data_scaler[col], color="Blue", shade=True)
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")
    ax = ax.legend(["train","test"])
 
plt.show()

#### 代码解释：
`for i, col in enumerate(test_data_scaler.columns)`：迭代循环，遍历`test_data_scaler.columns`中的每一列，并使用`enumerate`函数同时将列的索引赋值给`i`，列名赋值给`col`。


查看特征'V5', 'V17', 'V28', 'V22', 'V11', 'V9'数据的数据分布

In [ ]:
drop_col = 6
drop_row = 1

plt.figure(figsize=(5*drop_col,5*drop_row))

for i, col in enumerate(["V5","V9","V11","V17","V22","V28"]):
    ax =plt.subplot(drop_row,drop_col,i+1)
    ax = sns.kdeplot(train_data_scaler[col], color="Red", shade=True)
    ax= sns.kdeplot(test_data_scaler[col], color="Blue", shade=True)
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")
    ax = ax.legend(["train","test"])
plt.show()

这几个特征下，训练集的数据和测试集的数据分布不一致，会影响模型的泛化能力，故删除这些特征

In [ ]:
data_train_scaler = train_data_scaler.drop(['V5','V9','V11','V17','V22','V28'],axis=1)
data_train_scaler.head()

## 4.4特征相关性

In [ ]:
plt.figure(figsize=(20, 16))  
column = train_data_scaler.columns.tolist()  
mcorr = train_data_scaler[column].corr(method="spearman")  
mask = np.zeros_like(mcorr, dtype=np.bool)  
mask[np.triu_indices_from(mask)] = True  
cmap = sns.diverging_palette(220, 10, as_cmap=True)  
g = sns.heatmap(mcorr, mask=mask, cmap=cmap, square=True, annot=True, fmt='0.2f')  
plt.show()

## 4.5特征降维

### 4.5.1相关性初筛1
进行特征相关性初筛，计算相关性系数并筛选大于0.1的特征变量

In [ ]:
mcorr=mcorr.abs()
numerical_corr=mcorr[mcorr['target']>0.1]['target']
print(numerical_corr.sort_values(ascending=False))

# 还可以使用spearman相关性系数进行排序显示，如下：
index0 = numerical_corr.sort_values(ascending=False).index
print(train_data_scaler[index0].corr('spearman'))

#### 代码详解：

1. 使用 `mcorr.abs()`对相关性矩阵 `mcorr` 中的各个元素取绝对值。


2. `mcorr[mcorr['target']>0.1]` 从 `mcorr` 中筛选出目标变量（`target`）相关性大于 0.1 的特征行，接着 `mcorr[mcorr['target']>0.1]['target']`提取出 `target` 列。


3. 对 `numerical_corr` 进行降序排序，`.sort_values(ascending=False)` 表示按照降序排列，即相关性系数较高的特征显示在前面。


4. `numerical_corr.sort_values(ascending=False).index` 对 `numerical_corr` 进行降序排序，并提取排序后的索引，并将其赋值给 `index0`。


5. 通过 `train_data_scaler[index0]` 获取 `train_data_scaler` 中与 `index0` 中特征名称索引对应的特征列，并使用 `.corr('spearman')` 计算这些特征之间的斯皮尔曼相关系数。


### 4.5.2相关性初筛2

In [ ]:
features_corr = numerical_corr.sort_values(ascending=False).reset_index()
features_corr.columns = ['features_and_target', 'corr']
features_corr_select = features_corr[features_corr['corr']>0.3] # 筛选出大于相关性大于0.3的特征
print(features_corr_select)
select_features = [col for col in features_corr_select['features_and_target'] if col not in ['target']]
new_train_data_corr_select = train_data_scaler[select_features+['target']]
new_test_data_corr_select = test_data_scaler[select_features]

## 4.6多重共线性分析
多重共线性分析的原则是特征组之间的相关性系数较大，即每个特征变量与其他特征变量之间的相关性系数较大，故可能存在较大的共线性影响，这会导致模型估计不准确。因此，后续要使用PCA对数据进行处理，去除多重共线性。

In [ ]:
# 计算多重共线性
from statsmodels.stats.outliers_influence import variance_inflation_factor #多重共线性方差膨胀因子

#多重共线性
new_numerical=['V0', 'V2', 'V3', 'V4', 'V5', 'V6', 'V10','V11', 
                         'V13', 'V15', 'V16', 'V18', 'V19', 'V20', 'V22','V24','V30', 'V31', 'V37']
X=np.matrix(train_data_scaler[new_numerical])
VIF_list=[variance_inflation_factor(X, i) for i in range(X.shape[1])]
VIF_list

说明：new_numerical的特征变量可以根据相关性矩阵看出并过滤筛选，剔除那些与其他特征高度相关的特征，然后，使用剩余的特征矩阵来计算每个特征的VIF值，以评估自变量之间的多重共线性程度。

#### 代码详解：

1. `new_numerical=[ ]`：定义了一个包含特征列名称的列表`new_numerical`，这些特征将被用于计算VIF。


3. `X=np.matrix(train_data_scaler[new_numerical])`：将训练数据`train_data_scaler`中`new_numerical`列表中的特征列用 `np.matrix` 转换为二维矩阵`X`。该矩阵将作为VIF计算的输入。


4. `VIF_list=[variance_inflation_factor(X, i) for i in range(X.shape[1])]`：使用列表推导式计算VIF列表。对于矩阵`X`的每一列（即特征），调用`variance_inflation_factor`函数来计算VIF，并将结果添加到`VIF_list`中。`range(X.shape[1])`生成了一个表示列索引的迭代器。（下方详解variance_inflation_factor）


5. `VIF_list`：输出VIF列表，该列表包含了每个特征的方差膨胀因子。

----
**variance_inflation_factor用法**

`variance_inflation_factor` 是一个用于计算方差膨胀因子（Variance Inflation Factor，VIF）的函数。方差膨胀因子是用于检测多重共线性（multicollinearity）问题的一种统计指标。

多重共线性指的是在回归分析中，自变量之间存在高度线性相关性的情况。它会导致回归模型中的估计不稳定，使得对自变量的解释变得困难。方差膨胀因子通过衡量每个自变量对其他自变量的线性相关性来检测多重共线性。

`variance_inflation_factor` 函数通常需要传入两个参数：

1. `X`：一个二维数组或数据帧，表示自变量矩阵。每一列代表一个自变量。
2. `idx`：一个整数，表示要计算方差膨胀因子的自变量的索引。

以下是 `variance_inflation_factor` 函数的用法示例：

```python
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 假设 X 是自变量矩阵，其中每一列代表一个自变量
vif = variance_inflation_factor(X, idx)
print(f"Variance inflation factor for variable {idx}: {vif}")
```

这段代码计算自变量矩阵 `X` 中给定索引 `idx` 的自变量的方差膨胀因子，并将结果打印输出。

方差膨胀因子的常见阈值为 5 或 10。当方差膨胀因子超过这个阈值时，表示该自变量与其他自变量存在高度线性相关性，可能会导致多重共线性问题。一般而言，方差膨胀因子越大，说明自变量之间的相关性越高。


## 4.7PCA处理
利用PCA方法去除数据的多重共线性，并进行降维。

在scikit-learn（sklearn）库中，PCA被实现`sklearn.decomposition.PCA`类，主要参数`n_components`指定降维后的特征数量或保留的信息量的比例。可以设置为一个整数来指定具体的特征数量，也可以设置为一个0到1之间的浮点数来表示保留的信息量比例。

In [ ]:
from sklearn.decomposition import PCA   #主成分分析法

#PCA方法降维
#保持90%的信息
pca = PCA(n_components=0.9)
new_train_pca_90 = pca.fit_transform(train_data_scaler.iloc[:,0:-1])
new_test_pca_90 = pca.transform(test_data_scaler)

new_train_pca_90 = pd.DataFrame(new_train_pca_90)
new_test_pca_90 = pd.DataFrame(new_test_pca_90)

new_train_pca_90['target'] = train_data_scaler['target']

# 查看原数据信息并与PCA处理后的数据进行对比
display(new_train_pca_90.describe())
display(train_data_scaler.describe())

In [ ]:
#PCA方法降维
#保留16个主成分
pca = PCA(n_components=16)
new_train_pca_16 = pca.fit_transform(train_data_scaler.iloc[:,0:-1])
new_test_pca_16 = pca.transform(test_data_scaler)
new_train_pca_16 = pd.DataFrame(new_train_pca_16)
new_test_pca_16 = pd.DataFrame(new_test_pca_16)
new_train_pca_16['target'] = train_data_scaler['target']

display(new_train_pca_16.describe())
display(new_test_pca_16.describe())

# 线性回归

## 导入相关库

In [2]:
from sklearn.linear_model import LinearRegression  #线性回归
from sklearn.neighbors import KNeighborsRegressor  #K近邻回归
from sklearn.tree import DecisionTreeRegressor     #决策树回归
from sklearn.ensemble import RandomForestRegressor #随机森林回归
from sklearn.svm import SVR  #支持向量回归
import lightgbm as lgb #lightGbm模型
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split # 切分数据
from sklearn.metrics import mean_squared_error #评价指标

ModuleNotFoundError: No module named 'sklearn.linear_model'

## 切分数据集

In [ ]:
#采用 pca 保留16维特征的数据
new_train_pca_16 = new_train_pca_16.fillna(0)
train = new_train_pca_16[new_test_pca_16.columns]
target = new_train_pca_16['target']

# 切分数据 训练数据80% 验证数据20%
train_data,test_data,train_target,test_target=train_test_split(train,target,test_size=0.2,random_state=0)

## 多元线性回归模型

In [ ]:
clf = LinearRegression()
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("LinearRegression:   ", score)

In [ ]:
train_score = []
test_score = []

# 给予不同的数据量，查看模型的学习效果
for i in range(10, len(train_data)+1, 10):
    lin_reg = LinearRegression()
    lin_reg.fit(train_data[:i], train_target[:i])
    # LinearRegression().fit(X_train[:i], y_train[:i])
    
    # 查看模型的预测情况：两种，模型基于训练数据集预测的情况(可以理解为模型拟合训练数据集的情况)，模型基于测试数据集预测的情况
    # 此处使用 lin_reg.predict(X_train[:i])，为训练模型的全部数据集
    y_train_predict = lin_reg.predict(train_data[:i])
    train_score.append(mean_squared_error(train_target[:i], y_train_predict))
    
    y_test_predict = lin_reg.predict(test_data)
    test_score.append(mean_squared_error(test_target, y_test_predict))
    
# np.sqrt(train_score)：将列表 train_score 中的数开平方
plt.plot([i for i in range(1, len(train_score)+1)], train_score, label='train')
plt.plot([i for i in range(1, len(test_score)+1)], test_score, label='test')

# plt.legend()：显示图例（如图形的 label）；
plt.legend()
plt.show()

In [ ]:
def plot_learning_curve(algo, X_train, X_test, y_train, y_test):
    """绘制学习曲线：只需要传入算法(或实例对象)、X_train、X_test、y_train、y_test"""
    """当使用该函数时传入算法，该算法的变量要进行实例化，如：PolynomialRegression(degree=2)，变量 degree 要进行实例化"""
    train_score = []
    test_score = []
    for i in range(10, len(X_train)+1, 10):
        algo.fit(X_train[:i], y_train[:i])
        
        y_train_predict = algo.predict(X_train[:i])
        train_score.append(mean_squared_error(y_train[:i], y_train_predict))
    
        y_test_predict = algo.predict(X_test)
        test_score.append(mean_squared_error(y_test, y_test_predict))
    
    plt.plot([i for i in range(1, len(train_score)+1)],
            train_score, label="train")
    plt.plot([i for i in range(1, len(test_score)+1)],
            test_score, label="test")
    
    plt.legend()
    plt.show()

In [ ]:
#plot_learning_curve(LinearRegression(), train_data, test_data, train_target, test_target)

## K近邻回归

In [ ]:
for i in range(3,20):
    clf = KNeighborsRegressor(n_neighbors=i) # 最近三个
    clf.fit(train_data, train_target)
    score = mean_squared_error(test_target, clf.predict(test_data))
    print("KNeighborsRegressor:   ", score)

In [ ]:
#plot_learning_curve(KNeighborsRegressor(n_neighbors=5) , train_data, test_data, train_target, test_target)

## 决策树回归

In [ ]:
clf = DecisionTreeRegressor() 
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("DecisionTreeRegressor:   ", score)

In [ ]:
#plot_learning_curve(DecisionTreeRegressor(), train_data, test_data, train_target, test_target)

## 随机森林回归

In [ ]:
clf = RandomForestRegressor(n_estimators=200) # 200棵树模型
clf.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("RandomForestRegressor:   ", score)

In [ ]:
#plot_learning_curve(RandomForestRegressor(n_estimators=200), train_data, test_data, train_target, test_target)

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

myGBR = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                                  learning_rate=0.03, loss='huber', max_depth=14,
                                  max_features='sqrt', max_leaf_nodes=None,
                                  min_impurity_decrease=0.0, min_impurity_split=None,
                                  min_samples_leaf=10, min_samples_split=40,
                                  min_weight_fraction_leaf=0.0, n_estimators=300,
                                  presort='auto', random_state=10, subsample=0.8, verbose=0,
                                  warm_start=False)
myGBR.fit(train_data, train_target)
score = mean_squared_error(test_target, clf.predict(test_data))
print("GradientBoostingRegressor:   ", score)


In [ ]:
myGBR = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                                  learning_rate=0.03, loss='huber', max_depth=14,
                                  max_features='sqrt', max_leaf_nodes=None,
                                  min_impurity_decrease=0.0, min_impurity_split=None,
                                  min_samples_leaf=10, min_samples_split=40,
                                  min_weight_fraction_leaf=0.0, n_estimators=300,
                                  presort='auto', random_state=10, subsample=0.8, verbose=0,
                                  warm_start=False)

#plot_learning_curve(myGBR, train_data, test_data, train_target, test_target)